In [103]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from bs4 import BeautifulSoup
import re
import string

In [104]:
pip install beautifulsoup4

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [105]:
df = pd.read_csv('/Users/youziya/Library/Group Containers/UBF8T346G9.OneDriveStandaloneSuite/OneDrive - York University.noindex/OneDrive - York University/MBAN 6090 Analytics Project/jobs_info.csv')

In [106]:
df.head()

,Unnamed: 0,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description
0,1,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","['Media', 'Software']",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w..."
1,2,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","['Information Technology', 'Media', 'Sales and...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st..."
2,3,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",['Agriculture and Farming'],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w..."
3,4,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","['Education', 'Information Technology', 'Softw...",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax..."
4,5,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","['Biotechnology', 'DeepTech', 'Health', 'Softw...",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b..."


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  4182 non-null   int64 
 1   organization_name           4182 non-null   object
 2   title                       4182 non-null   object
 3   locations                   4182 non-null   object
 4   organization_topics         4182 non-null   object
 5   organization_industry_tags  4182 non-null   object
 6   url                         4182 non-null   object
 7   has_description             4182 non-null   bool  
 8   description                 4159 non-null   object
dtypes: bool(1), int64(1), object(7)
memory usage: 265.6+ KB


In [108]:
df = df.drop(columns=['Unnamed: 0'])


In [109]:
import pandas as pd

# Assuming df is your DataFrame

# Define a parsing function for list-based locations
def parse_location_list(location_list):
    # Flatten the list into a single string separated by commas
    location_str = ', '.join(location_list)
    
    # Now you can use the same parsing logic as before
    city, state_province, country, is_remote = '', '', '', False

    # Check for 'Remote'
    if 'Remote' in location_str:
        is_remote = True

    # Split the location string
    parts = location_str.replace('Remote', '').split(', ')
    parts = [part for part in parts if part]  # Remove empty parts

    # Update city, state_province, and country based on the number of parts
    if len(parts) >= 3:
        # Assuming the last part is always the country
        country = parts[-1]
        state_province = parts[-2]
        city = ', '.join(parts[:-2])  # Combine the rest as the city
    elif len(parts) == 2:
        state_province, country = parts
    elif len(parts) == 1:
        country = parts[0]

    return city, state_province, country, is_remote

# Apply the parsing function to each list in the 'locations' column
df[['City', 'State/Province', 'Country', 'Is_Remote']] = df['locations'].apply(lambda x: parse_location_list(x)).tolist()

# Display the result
print(df.head())


  organization_name                                              title  \
0           Vidyard          Director of Sales, Enterprise Acquisition   
1         Hootsuite  Senior Software Developer | Développeur logici...   
2            Semios                                    Chemistry Co-op   
3           Axonify  Senior Talent Acquisition Partner (15 months c...   
4     Deep Genomics  (Senior) Research Scientist - Statistical Gene...   

                                          locations  \
0             ['United States', 'Canada', 'Remote']   
1  ['Toronto, ON, Canada', 'Vancouver, BC, Canada']   
2                         ['Vancouver, BC, Canada']   
3   ['Toronto, ON, Canada', 'Waterloo, ON, Canada']   
4                           ['Toronto, ON, Canada']   

                                 organization_topics  \
0  ['B2B', 'Media & entertainment', 'Productivity...   
1  ['500+', 'Advertising/marketing', 'B2B', 'Soft...   
2  ['Agriculture', 'B2B', 'C100 alumni', 'Data & ...   


In [110]:
df.head()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","['Media', 'Software']",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...","[, ', U, n, i, t, e, d, , S, t, a, t, e, s, '...",',],False
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","['Information Technology', 'Media', 'Sales and...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",['Agriculture and Farming'],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...","[, ', V, a, n, c, o, u, v, e, r, ,, , B, C, ,...",',],False
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","['Education', 'Information Technology', 'Softw...",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","['Biotechnology', 'DeepTech', 'Health', 'Softw...",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False


In [111]:
df['locations'].unique()

array(["['United States', 'Canada', 'Remote']",
       "['Toronto, ON, Canada', 'Vancouver, BC, Canada']",
       "['Vancouver, BC, Canada']",
       "['Toronto, ON, Canada', 'Waterloo, ON, Canada']",
       "['Toronto, ON, Canada']", "['Canada']", "['Halifax, NS, Canada']",
       "['Oakville, ON, Canada']", "['Asia']", "['Centennial, CO, USA']",
       "['Charlottesville, VA, USA']", "['Montreal, QC, Canada']",
       "['New York, NY, USA', 'Boston, MA, USA', 'Chicago, IL, USA', 'Atlanta, GA, USA']",
       "['Mississauga, ON, Canada']",
       "['Remote', 'Mississauga, ON, Canada']", "['Sacramento, CA, USA']",
       "['Manila, Philippines']", "['Baltimore, MD, USA', 'Remote']",
       "['Bedford, NS, Canada']", "['United States', 'Remote']",
       "['United States']", '["St. John\'s, NL, Canada"]',
       "['Netherlands']", "['London, UK']", "['Austin, TX, USA']",
       "['Canada', 'Remote']", "['Edmonton, AB, Canada']",
       "['Berlin, Germany', 'Brandenburg, Germany']",
     

In [112]:
df.tail()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote
4177,Payment Source,hr@paymentsource.ca,"['Toronto, ON, Canada']","['B2B', 'Consumer & lifestyle', 'Finance', 'Sa...","['Ecommerce & Marketplace', 'Finance', 'Softwa...",https://www1.communitech.ca/companies/payment-...,True,"<div data-testid=""careerPage""><h2 class=""uk-he...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False
4178,Vantage,Intermediate/Senior Full Stack Developer,"['Toronto, ON, Canada']","['Advertising/marketing', 'B2B', 'C100 alumni'...","['Hardware', 'Sales and Marketing', 'Software']",https://www1.communitech.ca/companies/vantage/...,True,"<div data-testid=""careerPage""><div class=""cont...","[, ', T, o, r, o, n, t, o, ,, , O, N, ,, , C...",',],False
4179,StackAdapt,Data Scientist / Senior Data Scientist,['Canada'],"['Advertising/marketing', 'B2B', 'C100 alumni'...","['Sales and Marketing', 'Software']",https://www1.communitech.ca/companies/stackada...,True,"<div data-testid=""careerPage""><section><div>St...","[, ', C, a, n, a, d, a",',],False
4180,Replicon,Product Support Associate,"['India', 'Remote']","['500+', 'AI/ML', 'B2B', 'Productivity', 'SaaS...","['Information Technology', 'Software']",https://www1.communitech.ca/companies/replicon...,True,"<div data-testid=""careerPage""><div class=""cont...","[, ', I, n, d, i, a, ', ,, , ', R, e, m, o, t, e",',],False
4181,Replicon,Implementation Consultant,"['India', 'Remote']","['500+', 'AI/ML', 'B2B', 'Productivity', 'SaaS...","['Information Technology', 'Software']",https://www1.communitech.ca/companies/replicon...,True,"<div data-testid=""careerPage""><div class=""cont...","[, ', I, n, d, i, a, ', ,, , ', R, e, m, o, t, e",',],False


In [113]:
def parse_location(location_str):
    cities, provinces, countries = set(), set(), set()
    is_remote = False

    
    location_entries = location_str.strip("[]").split("', '")

    for entry in location_entries:
        entry = entry.strip("'\" ")
        if entry == 'Remote':
            is_remote = True
            continue

        parts = entry.split(', ')
        parts = [part.strip() for part in parts if part]  

   
        if len(parts) == 3:  
            city, province, country = parts
            cities.add(city)
            provinces.add(province)
            countries.add(country)
        elif len(parts) == 2:  
            first_part, second_part = parts
            if any(char.isdigit() for char in first_part) or len(first_part) == 2:
                provinces.add(first_part)
            else:
                cities.add(first_part)
            countries.add(second_part)
        elif len(parts) == 1:  
            countries.add(parts[0])

    return ', '.join(cities), ', '.join(provinces), ', '.join(countries), is_remote

# Apply the parsing function
df['City'], df['Province'], df['Country'], df['Is_Remote'] = zip(*df['locations'].apply(parse_location))

# View the result
df[['locations', 'City', 'Province', 'Country', 'Is_Remote']]


,locations,City,Province,Country,Is_Remote
0,"['United States', 'Canada', 'Remote']",,,"United States, Canada",True
1,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","Toronto, Vancouver","BC, ON",Canada,False
2,"['Vancouver, BC, Canada']",Vancouver,BC,Canada,False
3,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","Toronto, Waterloo",ON,Canada,False
4,"['Toronto, ON, Canada']",Toronto,ON,Canada,False
...,...,...,...,...,...
4177,"['Toronto, ON, Canada']",Toronto,ON,Canada,False
4178,"['Toronto, ON, Canada']",Toronto,ON,Canada,False
4179,['Canada'],,,Canada,False
4180,"['India', 'Remote']",,,India,True


In [114]:
df['Province'].unique()

array(['', 'BC, ON', 'BC', 'ON', 'NS', 'CO', 'VA', 'QC', 'NY, MA, GA, IL',
       'CA', 'MD', 'NL', 'TX', 'AB', 'AZ', 'state of são paulo', 'MB',
       'bc', 'NY', 'MA', 'NB', 'ON, NY', 'MN', 'DC', 'Delhi', 'İstanbul',
       'Haryana', 'Telangana', 'MO', 'ND', 'AK', 'MI', 'PE', 'Bogota',
       'KS', 'PA', 'Uttar Pradesh', 'GA', 'ON K0E 1K0', 'TN', 'WA',
       'PA, CA', 'Karnataka', 'LA', 'London', 'FL', 'MB, ON, BC', 'UT',
       'SK', 'Tamil Nadu', 'Santiago Metropolitan Region', 'qc', 'IL',
       'NH', 'nb', 'OH', 'ON, CA', 'MB, ON', 'Punjab', 'NJ',
       'NV, AZ, OR, CA', 'West Bengal', 'Gujarat', 'NC', 'MS', 'Kerala',
       'NY, MA, GA', 'NM', 'Santa Fe Province', 'VT', 'ON, bc', 'RJ',
       'MG', 'SP', 'BA', 'TX, CA', 'Maharashtra', 'QC, ON, BC',
       'NY, TX, IL', 'BC, bc', 'QC, ON', 'AL', 'ON N0G',
       'Kanagawa Prefecture', 'TX, AB', 'Baja California', 'Nuevo Leon',
       'Jalisco', 'MA, NS', 'NB, VA', 'QC, TX'], dtype=object)

In [115]:
df['organization_industry_tags']

0                                   ['Media', 'Software']
1       ['Information Technology', 'Media', 'Sales and...
2                             ['Agriculture and Farming']
3       ['Education', 'Information Technology', 'Softw...
4       ['Biotechnology', 'DeepTech', 'Health', 'Softw...
                              ...                        
4177    ['Ecommerce & Marketplace', 'Finance', 'Softwa...
4178      ['Hardware', 'Sales and Marketing', 'Software']
4179                  ['Sales and Marketing', 'Software']
4180               ['Information Technology', 'Software']
4181               ['Information Technology', 'Software']
Name: organization_industry_tags, Length: 4182, dtype: object

In [116]:
df['City']

0                         
1       Toronto, Vancouver
2                Vancouver
3        Toronto, Waterloo
4                  Toronto
               ...        
4177               Toronto
4178               Toronto
4179                      
4180                      
4181                      
Name: City, Length: 4182, dtype: object

In [117]:
df.dropna()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,Country,Is_Remote,Province
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","['Media', 'Software']",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",,',"United States, Canada",True,
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","['Information Technology', 'Media', 'Sales and...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, Vancouver",',Canada,False,"BC, ON"
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",['Agriculture and Farming'],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,',Canada,False,BC
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","['Education', 'Information Technology', 'Softw...",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, Waterloo",',Canada,False,ON
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","['Biotechnology', 'DeepTech', 'Health', 'Softw...",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,',Canada,False,ON
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,Payment Source,hr@paymentsource.ca,"['Toronto, ON, Canada']","['B2B', 'Consumer & lifestyle', 'Finance', 'Sa...","['Ecommerce & Marketplace', 'Finance', 'Softwa...",https://www1.communitech.ca/companies/payment-...,True,"<div data-testid=""careerPage""><h2 class=""uk-he...",Toronto,',Canada,False,ON
4178,Vantage,Intermediate/Senior Full Stack Developer,"['Toronto, ON, Canada']","['Advertising/marketing', 'B2B', 'C100 alumni'...","['Hardware', 'Sales and Marketing', 'Software']",https://www1.communitech.ca/companies/vantage/...,True,"<div data-testid=""careerPage""><div class=""cont...",Toronto,',Canada,False,ON
4179,StackAdapt,Data Scientist / Senior Data Scientist,['Canada'],"['Advertising/marketing', 'B2B', 'C100 alumni'...","['Sales and Marketing', 'Software']",https://www1.communitech.ca/companies/stackada...,True,"<div data-testid=""careerPage""><section><div>St...",,',Canada,False,
4180,Replicon,Product Support Associate,"['India', 'Remote']","['500+', 'AI/ML', 'B2B', 'Productivity', 'SaaS...","['Information Technology', 'Software']",https://www1.communitech.ca/companies/replicon...,True,"<div data-testid=""careerPage""><div class=""cont...",,',India,True,


In [118]:
def consolidate_job_titles(job_title):
    # Define broad categories with keywords
    categories = {
        'Director': ['Director'],
        'Senior': ['Senior'],
        'Co-op': ['Co-op']
    }

    # Default category
    consolidated_category = 'Other'

    # Check for each keyword in the job title
    for category, keywords in categories.items():
        if any(keyword in job_title for keyword in keywords):
            consolidated_category = category
            break

    return consolidated_category

# Apply the function to the 'title' column
df['consolidatedJobTitle'] = df['title'].apply(consolidate_job_titles)

# Display the first few rows to verify the changes
df[['title', 'consolidatedJobTitle']].head()


,title,consolidatedJobTitle
0,"Director of Sales, Enterprise Acquisition",Director
1,Senior Software Developer | Développeur logici...,Senior
2,Chemistry Co-op,Co-op
3,Senior Talent Acquisition Partner (15 months c...,Senior
4,(Senior) Research Scientist - Statistical Gene...,Senior


In [119]:
#presence 1 and absence 0
df['organization_industry_tags'] = df['organization_industry_tags'].apply(lambda x: ast.literal_eval(x))


mlb = MultiLabelBinarizer()
transformed_data = mlb.fit_transform(df['organization_industry_tags'])


industry_tags_df = pd.DataFrame(transformed_data, columns=mlb.classes_)


df_combined = pd.concat([df, industry_tags_df], axis=1)


df_combined.head()


,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,...,Real Estate,Sales and Marketing,Security,Software,Sports,Sustainability,Telecommunication,Transportation,Travel,Wellness and Beauty
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",,',...,0,0,0,1,0,0,0,0,0,0
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, Vancouver",',...,0,1,0,1,0,0,0,0,0,0
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,',...,0,0,0,0,0,0,0,0,0,0
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, Waterloo",',...,0,0,0,1,0,0,0,0,0,0
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,',...,0,0,0,1,0,0,0,0,0,0


In [120]:
def clean_html_description(text):

    if not isinstance(text, str):
        return ""  
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ')
    return cleaned_text

df['cleaned_description'] = df['description'].apply(clean_html_description)


In [121]:
df[['description', 'cleaned_description']].head()

,description,cleaned_description
0,"<div data-testid=""careerPage""><p>At Vidyard, w...","At Vidyard, we make life easier for sellers, m..."
1,"<div data-testid=""careerPage""><p><span><em><st...",Version française disponible ci-bas. We’re loo...
2,"<div data-testid=""careerPage""><p><strong>Who w...",Who we are: We are a bunch of people who reall...
3,"<div data-testid=""careerPage""><section><div>Ax...",Axonify is the frontline enablement solution t...
4,"<div data-testid=""careerPage""><section><div><b...",About Us: Deep Genomics is a startup company...


#### making them all lower case for NLM modelling, can remove it if no need

In [122]:
def further_clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split())

    return text

df['cleaned_description'] = df['cleaned_description'].apply(further_clean_text)
df[['description', 'cleaned_description']].head()


,description,cleaned_description
0,"<div data-testid=""careerPage""><p>At Vidyard, w...",at vidyard we make life easier for sellers mar...
1,"<div data-testid=""careerPage""><p><span><em><st...",version française disponible cibas we’re looki...
2,"<div data-testid=""careerPage""><p><strong>Who w...",who we are we are a bunch of people who really...
3,"<div data-testid=""careerPage""><section><div>Ax...",axonify is the frontline enablement solution t...
4,"<div data-testid=""careerPage""><section><div><b...",about us deep genomics is a startup company th...


In [123]:
df_combined.head()

,organization_name,title,locations,organization_topics,organization_industry_tags,url,has_description,description,City,State/Province,...,Real Estate,Sales and Marketing,Security,Software,Sports,Sustainability,Telecommunication,Transportation,Travel,Wellness and Beauty
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]",https://www1.communitech.ca/companies/vidyard/...,True,"<div data-testid=""careerPage""><p>At Vidyard, w...",,',...,0,0,0,1,0,0,0,0,0,0
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...",https://www1.communitech.ca/companies/hootsuit...,True,"<div data-testid=""careerPage""><p><span><em><st...","Toronto, Vancouver",',...,0,1,0,1,0,0,0,0,0,0
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],https://www1.communitech.ca/companies/semios/j...,True,"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,',...,0,0,0,0,0,0,0,0,0,0
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]",https://www1.communitech.ca/companies/axonify/...,True,"<div data-testid=""careerPage""><section><div>Ax...","Toronto, Waterloo",',...,0,0,0,1,0,0,0,0,0,0
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]",https://www1.communitech.ca/companies/deep-gen...,True,"<div data-testid=""careerPage""><section><div><b...",Toronto,',...,0,0,0,1,0,0,0,0,0,0


In [124]:
df_combined = df_combined.drop(columns=['has_description', 'State/Province','url'])

In [125]:
df_combined.head()

,organization_name,title,locations,organization_topics,organization_industry_tags,description,City,Country,Is_Remote,Province,...,Real Estate,Sales and Marketing,Security,Software,Sports,Sustainability,Telecommunication,Transportation,Travel,Wellness and Beauty
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","['B2B', 'Media & entertainment', 'Productivity...","[Media, Software]","<div data-testid=""careerPage""><p>At Vidyard, w...",,"United States, Canada",True,,...,0,0,0,1,0,0,0,0,0,0
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","['500+', 'Advertising/marketing', 'B2B', 'Soft...","[Information Technology, Media, Sales and Mark...","<div data-testid=""careerPage""><p><span><em><st...","Toronto, Vancouver",Canada,False,"BC, ON",...,0,1,0,1,0,0,0,0,0,0
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","['Agriculture', 'B2B', 'C100 alumni', 'Data & ...",[Agriculture and Farming],"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,Canada,False,BC,...,0,0,0,0,0,0,0,0,0,0
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","['B2B', 'Data & analytics', 'Education', 'Fema...","[Education, Information Technology, Software]","<div data-testid=""careerPage""><section><div>Ax...","Toronto, Waterloo",Canada,False,ON,...,0,0,0,1,0,0,0,0,0,0
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","['50-99', 'AI/ML', 'B2B', 'Biotechnology', 'Me...","[Biotechnology, DeepTech, Health, Software]","<div data-testid=""careerPage""><section><div><b...",Toronto,Canada,False,ON,...,0,0,0,1,0,0,0,0,0,0


In [132]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Now when you display your DataFrame, it should show all rows and columns
df_combined.info('include=all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 103 columns):
 #    Column                      Dtype 
---   ------                      ----- 
 0    organization_name           object
 1    title                       object
 2    locations                   object
 3    organization_topics         object
 4    organization_industry_tags  object
 5    description                 object
 6    City                        object
 7    Country                     object
 8    Is_Remote                   bool  
 9    Province                    object
 10   consolidatedJobTitle        object
 11   Agriculture and Farming     int64 
 12   Biotechnology               int64 
 13   Blockchain and Crypto       int64 
 14   Construction                int64 
 15   Consulting                  int64 
 16   Consumer Products           int64 
 17   DeepTech                    int64 
 18   ECommerce & Marketplace     int64 
 19   Ecommerce & Marketplace  

In [135]:
import pandas as pd
from collections import Counter
import ast

# Step 1: Preprocess and parse 'organization_topics'
def parse_topics(topics_str):
    try:
        # Assuming the topics are stored as a string representation of a list
        return ast.literal_eval(topics_str)
    except ValueError:
        # If parsing fails, return an empty list or handle as appropriate
        return []

df_combined['organization_topics'] = df_combined['organization_topics'].apply(parse_topics)

# Step 2: Count the frequency of each topic
topic_counts = Counter(topic for topics_list in df_combined['organization_topics'] for topic in topics_list)

# Step 3: Set a frequency threshold (e.g., a topic must appear in at least 5 records)
threshold = 5

# Step 4: Filter topics based on the threshold
filtered_topics = {topic for topic, count in topic_counts.items() if count >= threshold}

# Step 5: Create a binary representation for the filtered topics
for topic in filtered_topics:
    df_combined[topic] = df_combined['organization_topics'].apply(lambda topics: int(topic in topics))

df_combined.head()



/var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/ipykernel_9994/752152210.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[topic] = df_combined['organization_topics'].apply(lambda topics: int(topic in topics))
/var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/ipykernel_9994/752152210.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[topic] = df_combined['organization_topics'].apply(lambda topics: int(topic in topics))
/var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/ipykernel_9994/752152210.py:27: P

,organization_name,title,locations,organization_topics,organization_industry_tags,description,City,Country,Is_Remote,Province,consolidatedJobTitle,Agriculture and Farming,Biotechnology,Blockchain and Crypto,Construction,Consulting,Consumer Products,DeepTech,ECommerce & Marketplace,Ecommerce & Marketplace,Education,Energy,Fashion,Finance,Food,Gaming,Government and Military,HR and Recruiting,Hardware,Health,Information Technology,Internet Services,Manufacturing,Media,Music,Real Estate,Sales and Marketing,Security,Software,Sports,Sustainability,Telecommunication,Transportation,Travel,Wellness and Beauty,",",+,f,d,B,c,g,l,C,h,/,',H,e,n,4,P,b,M,G,0,r,y,a,&,),O,v,k,D,F,L,-,T,,],1,E,I,A,N,o,R,2,5,q,m,(,t,S,u,[,s,p,i,z,w,9,V,7,Infrastructure,Productivity,Growth,Gaming & esports,Robotics,500,Female founded,B2B,Oneeleven,Proptech,SaaS,Data & analytics,Marketplace,Pet/animal,11-50,Hospitality,Ecommerce/product,50-99,Agnostic (all),Service,Ecosystem organization,HR,Medical/life sciences,1-9,Cybersecurity,Aerospace,Mining,Patent,Developer tool,100-499,Team true north,Agriculture,C100 alumni,Healthcare & wellness,10-49,500+,Advertising/marketing,Food & Beverage,Logistics / supply chain,Hardware (non-robotics),Mobile & telecommunications,AI/ML,Social enterprise,Networks & social,Sensors/IoT,Crypto & blockchain,Semiconductors,Retail,Research,Ip/licensing,Nanotech & quantum,Industrial,Legal,Consumer & lifestyle,B2C,Media & entertainment,Deep tech
0,Vidyard,"Director of Sales, Enterprise Acquisition","['United States', 'Canada', 'Remote']","[B2B, Media & entertainment, Productivity, Saa...","[Media, Software]","<div data-testid=""careerPage""><p>At Vidyard, w...",,"United States, Canada",True,,Director,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,1,0,1,1,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Hootsuite,Senior Software Developer | Développeur logici...,"['Toronto, ON, Canada', 'Vancouver, BC, Canada']","[500+, Advertising/marketing, B2B, Software, T...","[Information Technology, Media, Sales and Mark...","<div data-testid=""careerPage""><p><span><em><st...","Toronto, Vancouver",Canada,False,"BC, ON",Senior,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,1,1,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Semios,Chemistry Co-op,"['Vancouver, BC, Canada']","[Agriculture, B2B, C100 alumni, Data & analyti...",[Agriculture and Farming],"<div data-testid=""careerPage""><p><strong>Who w...",Vancouver,Canada,False,BC,Co-op,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,1,0,1,0,0,1,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Axonify,Senior Talent Acquisition Partner (15 months c...,"['Toronto, ON, Canada', 'Waterloo, ON, Canada']","[B2B, Data & analytics, Education, Female foun...","[Education, Information Technology, Software]","<div data-testid=""careerPage""><section><div>Ax...","Toronto, Waterloo",Canada,False,ON,Senior,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,1,1,0,0,1,0,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Deep Genomics,(Senior) Research Scientist - Statistical Gene...,"['Toronto, ON, Canada']","[50-99, AI/ML, B2B, Biotechnology, Medical/lif...","[Biotechnology, DeepTech, Health, Software]","<div data-testid=""careerPage""><section><

In [128]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Columns: 103 entries, organization_name to 9
dtypes: bool(1), int64(92), object(10)
memory usage: 3.3+ MB
